# Project
**Jacob Medley**

The project should provide some useful functionality in science or engineering. It could be a command line utility, or a package to use in notebooks. There should be some substance, but it does not need to be extensive. I don't expect it should take more than a few hours to write the code. It is not necessary to write very sophisticated code. Overall the project should demonstrate you have learned something in this class.

1. The project must be pip installable
2. Your project should utilize git, and there should be a version history.
3. Your project should have some tests.
4. Your project should use at least one code quality tool.
5. Your project should have a readme.md and LICENSE file.
6. The code should be well documented.
7. The code should be original work.
8. You should push it to a GitHUB repo.

At the end of the mini you will give a live demonstration of your project and what it does.

 


---

The `SappyCSTR` package is designed to enhance the conversion of reactants in a Continuous Stirred Tank Reactor (CSTR) engaged in a saponification reaction. Utilizing the Python libraries `Pyomo` for mathematical optimization and `Thermo` for thermodynamic property calculations, this package focuses on optimizing key operational parameters such as inlet temperature, inlet concentration of reactants, reactor volume, and inlet volumetric flow rate. By providing a structured and efficient approach to adjust these parameters, `SappyCSTR` aims to achieve optimal reactant conversion, thereby enhancing the efficiency and output of chemical processes within CSTR systems.

Equations Governing the Reactor:  

$ \frac{dN_j}{dt} = F_{j, in} - F_{j, out} + r_j V$  

$\frac{dT}{dt} = \frac{\dot{Q} - \dot{W}_s - \sum F_{i,0} C_{P_i} (T - T_{i0}) + [- \Delta H_{Rx} (T)(-r_A V)]}{\sum N_i C_{P_i}}$  

In this package, the goal is that the optimal conditions will be for the CSTR at steady-state, so both of the above two equations will be set to 0.

The objective function to maximize is:  

$\text{X} = \frac{C_{A,in} - C_{A, out}}{C_{A, out}}$   

Assumptions:  

- Elementary kinetics, first order reaction.
- Shaft work is negligible. ($\dot{\text{W}_\text{s}}$ is 0).
- The reactor is adiabatic ($\dot{\text{Q}}$ is 0).
- Obtective is maximizing conversion of reactants. 
- The three variables to be optimized are inlet flow of each species ($F_{C_2 H_5 O_2 C_2 H_5,\;in}$   
  and $F_{\text{NaOH}, \; in}$), the volume of the overall reactor ($V$), and the inlet temperature of the reactor ($T_{in}$).

The following study was used as a reference for determining the rate constant, activation energy, and appropriate bounds for reactor conditions. The study cites literature values in other papers in addition to its own, making it a good source for estimating initial conditions and defining the optimization model. 

Source: https://www.researchgate.net/profile/Parashuram-Sahoo/publication/229360677_Kinetic_Studies_on_Saponification_of_Ethyl_Acetate_Using_an_Innovative_Conductivity-Monitoring_Instrument_with_a_Pulsating_Sensor/links/59ddcfa6a6fdcc276fb9231b/Kinetic-Studies-on-Saponification-of-Ethyl-Acetate-Using-an-Innovative-Conductivity-Monitoring-Instrument-with-a-Pulsating-Sensor.pdf#:~:text=URL%3A%20https%3A%2F%2Fwww.researchgate.net%2Fprofile%2FParashuram

In [ ]:
from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, Param, NonNegativeReals, value, exp
from thermo import Chemical

#Ethyl Acetate Saponification
def EASap(initial_conditions, rate_constant_303K, activation_energy, compound_names):
    model = ConcreteModel()

    def reaction_enthalpy(compound_names):
        delta_H_rxn_react = 0.0
        delta_H_rxn_prod = 0.0
        for compound in compound_names['reactants'].values():
            chemr = Chemical(compound['formula'])
            delta_H_rxn_react += compound['stoichiometry'] * chemr.Hf
        for compound in compound_names['products'].values():
            chemp = Chemical(compound['formula'])
            delta_H_rxn_prod += compound['stoichiometry'] * chemp.Hf
        return (delta_H_rxn_prod - delta_H_rxn_react)

    def specific_heat(compound_formula):
        compound = Chemical(compound_formula)
        compound.calculate(model.T())
        return compound.Cp

    R = 8.314  # Gas constant J/(mol K)
    model.v0 = Var(initialize=initial_conditions['v0'], within=NonNegativeReals, bounds=(0.01,1))
    model.C_A_in = Var(within=NonNegativeReals, initialize=initial_conditions['C_A_in'], bounds=(0,5))
    model.C_B_in = Var(within=NonNegativeReals, initialize=initial_conditions['C_B_in'],bounds=(0,5))
    model.C_A = Var(within=NonNegativeReals, initialize=model.C_A_in,bounds=(0,5)) #Concentration of A
    model.C_B = Var(within=NonNegativeReals, initialize=model.C_B_in,bounds=(0,5))
    model.V = Var(initialize=initial_conditions['V'], within=NonNegativeReals,bounds=(0.1, 10))
    model.T_in = Var(initialize=303, within=NonNegativeReals, bounds=(300, 1000))
    model.T = Var(within=NonNegativeReals, initialize=model.T_in,bounds=(300, 1000))
    model.T_ref = Param(initialize=303)

    model.C_C_in = Param(initialize=0)
    model.C_D_in = Param(initialize=0)
    model.C_C = Var(within=NonNegativeReals, initialize=0)
    model.C_D = Var(within=NonNegativeReals, initialize=0)
    model.Q_dot = Param(initialize=0)
    model.W_dot = Param(initialize=0)
    model.k0 = Param(initialize=rate_constant_303K)
    model.Ea = Param(initialize=activation_energy)
    model.k = model.k0 * exp(-model.Ea / (R * model.T))

    model.reaction_rate = Var()
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == model.k * model.C_A * model.C_B)

    def energy_balance_rule(model):
        Cp_A = specific_heat(compound_names['reactants']['A']['formula'])
        Cp_B = specific_heat(compound_names['reactants']['B']['formula'])
        Cp_C = specific_heat(compound_names['products']['C']['formula'])
        Cp_D = specific_heat(compound_names['products']['D']['formula'])
        heat_capacity_flow_inlet = (model.C_A_in * Cp_A + model.C_B_in * Cp_B) * (model.T - model.T_in)
        heat_capacity_flow_outlet = (model.C_C * Cp_C + model.C_D * Cp_D) * (model.T - model.T_ref)
        reaction_enthalpy_term = reaction_enthalpy(compound_names) * model.reaction_rate * model.V
        return (model.Q_dot - model.W_dot - heat_capacity_flow_inlet + heat_capacity_flow_outlet + reaction_enthalpy_term) == 0

    model.energy_balance = Constraint(rule=energy_balance_rule)

    def mole_balance_ruleA(model):
        return model.C_A_in * model.v0 - model.C_A * model.v0 - model.reaction_rate * model.V == 0
    model.mole_balance_A = Constraint(rule=mole_balance_ruleA)

    def mole_balance_ruleB(model):
        return model.C_B_in * model.v0 - model.C_B * model.v0 - model.reaction_rate * model.V == 0
    model.mole_balance_B = Constraint(rule=mole_balance_ruleB)

    def mole_balance_ruleC(model):
        return model.C_C_in * model.v0 - model.C_C * model.v0 + model.reaction_rate * model.V == 0
    model.mole_balance_C = Constraint(rule=mole_balance_ruleC)

    def mole_balance_ruleD(model):
        return model.C_D_in * model.v0 - model.C_D * model.v0 + model.reaction_rate * model.V == 0
    model.mole_balance_D = Constraint(rule=mole_balance_ruleD)

    def objective_rule(model):
        return (model.C_A_in - model.C_A) / model.C_A_in
    model.objective = Objective(rule=objective_rule, sense=1)  # Maximize conversion

    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('C_A_in:', value(model.C_A_in), 'M')
        print('C_B_in:', value(model.C_B_in), 'M')
        print('V:', value(model.V), 'L')
        print('v0:', value(model.v0), 'L/s')
        print('T_in:', value(model.T_in), 'K')
    else:
        print('No optimal solution found.')

    return model


compound_names = {
    'reactants': {
        'A': {'formula': 'C4H8O2', 'stoichiometry': 1},  # Ethyl Acetate
        'B': {'formula': 'NaOH', 'stoichiometry': 1},     # Sodium Hydroxide
    },
    'products': {
        'C': {'formula': 'C2H5OH', 'stoichiometry': 1},  # Ethanol
        'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},   # Sodium Acetate
    }
}

# Example usage of the optimization function

#Initial conditions are based on laboratory conditions
initial_conditions = {
    'C_A_in': 1.0,  # Inlet Concentration of A (M)
    'C_B_in': 1.0,  # Inlet Concentration of B (M)
     'v0': 0.5,# volumwtric flow rate (L/s)
     'V': 5, # Reactor volume (L)
}
rate_constant_303K = 0.18 # Example rate constant at 303K for ESTERIFICATION
activation_energy = 48.4*1000  # Example activation energy in J/mol for ESTERIFICATION

# Call the function
model = EASap(initial_conditions, rate_constant_303K, activation_energy, compound_names)

# Building the Package

### Setting Up the Package Structure

In [ ]:
!mkdir -p SappyCSTR


!mkdir -p SappyCSTR/SappyCSTR
!mkdir -p SappyCSTR/tests

!touch SappyCSTR/SappyCSTR/__init__.py
!touch SappyCSTR/tests/__init__.py


!touch SappyCSTR/SappyCSTR/core.py
!touch SappyCSTR/tests/test_core.py


!touch SappyCSTR/setup.py
!touch SappyCSTR/requirements.txt
!touch SappyCSTR/README.md
!touch SappyCSTR/LICENSE


### Writing the Package's `core.py` File

In [ ]:
core_py_func = """

from pyomo.environ import ConcreteModel, Var, Constraint, Objective, SolverFactory, Param, NonNegativeReals, value, exp
from thermo import Chemical

def EASap(initial_conditions, rate_constant_303K, activation_energy, compound_names):

    model = ConcreteModel()
    def reaction_enthalpy(compound_names):
        delta_H_rxn_react = 0.0
        delta_H_rxn_prod = 0.0
        for compound in compound_names['reactants'].values():
            chemr = Chemical(compound['formula'])
            delta_H_rxn_react += compound['stoichiometry'] * chemr.Hf
        for compound in compound_names['products'].values():
            chemp = Chemical(compound['formula'])
            delta_H_rxn_prod += compound['stoichiometry'] * chemp.Hf
        return (delta_H_rxn_prod - delta_H_rxn_react)

    def specific_heat(compound_formula):
        compound = Chemical(compound_formula)
        compound.calculate(model.T())
        return compound.Cp

    R = 8.314
    model.v0 = Var(initialize=initial_conditions['v0'], within=NonNegativeReals, bounds=(0.01, 1))
    model.C_A_in = Var(within=NonNegativeReals, initialize=initial_conditions['C_A_in'], bounds=(0, 5))
    model.C_B_in = Var(within=NonNegativeReals, initialize=initial_conditions['C_B_in'], bounds=(0, 5))
    model.C_A = Var(within=NonNegativeReals, initialize=model.C_A_in, bounds=(0, 5))
    model.C_B = Var(within=NonNegativeReals, initialize=model.C_B_in, bounds=(0, 5))
    model.V = Var(initialize=initial_conditions['V'], within=NonNegativeReals, bounds=(0.1, 10))
    model.T_in = Var(initialize=303, within=NonNegativeReals, bounds=(300, 1000))
    model.T = Var(within=NonNegativeReals, initialize=model.T_in, bounds=(300, 1000))
    model.T_ref = Param(initialize=303)
    model.C_C_in = Param(initialize=0)
    model.C_D_in = Param(initialize=0)
    model.C_C = Var(within=NonNegativeReals, initialize=0)
    model.C_D = Var(within=NonNegativeReals, initialize=0)
    model.Q_dot = Param(initialize=0)
    model.W_dot = Param(initialize=0)
    model.k0 = Param(initialize=rate_constant_303K)
    model.Ea = Param(initialize=activation_energy)
    model.k = model.k0 * exp(-model.Ea / (R * model.T))
    model.reaction_rate = Var()
    model.reaction_rate_constraint = Constraint(expr=model.reaction_rate == model.k * model.C_A * model.C_B)

    def energy_balance_rule(model):
        Cp_A = specific_heat(compound_names['reactants']['A']['formula'])
        Cp_B = specific_heat(compound_names['reactants']['B']['formula'])
        Cp_C = specific_heat(compound_names['products']['C']['formula'])
        Cp_D = specific_heat(compound_names['products']['D']['formula'])
        heat_capacity_flow_inlet = (model.C_A_in * Cp_A + model.C_B_in * Cp_B) * (model.T - model.T_in)
        heat_capacity_flow_outlet = (model.C_C * Cp_C + model.C_D * Cp_D) * (model.T - model.T_ref)
        reaction_enthalpy_term = reaction_enthalpy(compound_names) * model.reaction_rate * model.V
        return (model.Q_dot - model.W_dot - heat_capacity_flow_inlet + heat_capacity_flow_outlet + reaction_enthalpy_term) == 0

    model.energy_balance = Constraint(rule=energy_balance_rule)

    def mole_balance_ruleA(model):
        return model.C_A_in * model.v0 - model.C_A * model.v0 - model.reaction_rate * model.V == 0
    model.mole_balance_A = Constraint(rule=mole_balance_ruleA)

    def mole_balance_ruleB(model):
        return model.C_B_in * model.v0 - model.C_B * model.v0 - model.reaction_rate * model.V == 0
    model.mole_balance_B = Constraint(rule=mole_balance_ruleB)

    def mole_balance_ruleC(model):
        return model.C_C_in * model.v0 - model.C_C * model.v0 + model.reaction_rate * model.V == 0
    model.mole_balance_C = Constraint(rule=mole_balance_ruleC)

    def mole_balance_ruleD(model):
        return model.C_D_in * model.v0 - model.C_D * model.v0 + model.reaction_rate * model.V == 0
    model.mole_balance_D = Constraint(rule=mole_balance_ruleD)

    def objective_rule(model):
        return (model.C_A_in - model.C_A) / model.C_A_in
    model.objective = Objective(rule=objective_rule, sense=1)

    solver = SolverFactory('ipopt')
    results = solver.solve(model)

    if (results.solver.status == 'ok') and (results.solver.termination_condition == 'optimal'):
        print('Found optimal solution.')
        print('C_A_in:', value(model.C_A_in), 'M')
        print('C_B_in:', value(model.C_B_in), 'M')
        print('V:', value(model.V), 'L')
        print('v0:', value(model.v0), 'L/s')
        print('T_in:', value(model.T_in), 'K')
    else: 
        print('No optimal solution found.')

    return model
"""

core_file_path = '/Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/SappyCSTR/core.py'

# Write the code_content to core.py
with open(core_file_path, 'w') as file:
    file.write(core_py_func)

### Writing the `setup.py` File Using the `setuptools` Library

In [ ]:
%%bash
cd /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/

In [ ]:
setup_code = """
from setuptools import setup, find_packages

setup(
    name='SappyCSTR',
    version='0.1.0',
    packages=find_packages(),
    install_requires=[
        'pyomo',
        'thermo'
    ],
    author='Jacob Medley',
    author_email='jmedley@andrew.cmu.edu',
    description='A package to optimize CSTRs for saponification reactions.',
    url='https://github.com/jmedjpg/optimize_CSTR',
    classifiers=[
        'Development Status :: 3 - Alpha',
        'Intended Audience :: Developers',
        'License :: OSI Approved :: MIT License',
        'Programming Language :: Python :: 3',
        'Programming Language :: Python :: 3.10',
        'Programming Language :: Python :: 3.11',
    ],
)
"""

with open('/Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/setup.py', 'w') as f:
    f.write(setup_code)

### Writting a Test Function for the Package  

In this test, we will analyze the saponification reaction between Ethyl Acetate and Sodium Hydroxide, which yields Ethanol and Sodium Acetate, as illustrated in the initial code example. We have optimized the reaction conditions using specified initial parameters and compound nomenclature that align with those detailed in the research study referenced. The configuration of the model.Var() and model.Param() arguments draws directly from the experimental procedures outlined in the cited paper, ensuring that our simulation closely mirrors the empirical conditions presented in the original research. While direct verification of the results for the precise inputs remains beyond our reach, the outcomes appear to be consistent with the expectations set forth in the research paper, showing reasonable agreement with the reported data.

Source: https://www.researchgate.net/profile/Parashuram-Sahoo/publication/229360677_Kinetic_Studies_on_Saponification_of_Ethyl_Acetate_Using_an_Innovative_Conductivity-Monitoring_Instrument_with_a_Pulsating_Sensor/links/59ddcfa6a6fdcc276fb9231b/Kinetic-Studies-on-Saponification-of-Ethyl-Acetate-Using-an-Innovative-Conductivity-Monitoring-Instrument-with-a-Pulsating-Sensor.pdf#:~:text=URL%3A%20https%3A%2F%2Fwww.researchgate.net%2Fprofile%2FParashuram

In [ ]:
%%bash
cat > /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/tests/test_core.py <<EOL
import pytest
from pyomo.environ import value
from SappyCSTR.core import EASap  # Make sure to import your function

def test_optimize_cstr():
    initial_conditions = {
        'C_A_in': 1.0,  # Inlet Concentration of A (M)
        'C_B_in': 1.0,  # Inlet Concentration of B (M)
        'v0': 0.5,  # volumetric flow rate (L/s)
        'V': 5,  # Reactor volume (L)
    }
    rate_constant_303K = 0.18  # Example rate constant at 303K for ESTERIFICATION
    activation_energy = 48.4*1000  # Example activation energy in J/mol for ESTERIFICATION
    compound_names = {
        'reactants': {
            'A': {'formula': 'C4H8O2', 'stoichiometry': 1},  # Ethyl Acetate
            'B': {'formula': 'NaOH', 'stoichiometry': 1},     # Sodium Hydroxide
        },
        'products': {
            'C': {'formula': 'C2H5OH', 'stoichiometry': 1},  # Ethanol
            'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},   # Sodium Acetate
        }
    }

    model = optimize_cstr(initial_conditions, rate_constant_303K, activation_energy, compound_names)

    assert value(model.C_A_in) == pytest.approx(3.1170817473657824, 0.01)
    assert value(model.C_B_in) == pytest.approx(1.5222873831305916, 0.01)
    assert value(model.V) == pytest.approx(0.6234563738047896, 0.01)
    assert value(model.v0) == pytest.approx(0.6814717895958581, 0.01)
    assert value(model.T_in) == pytest.approx(332.0706816612416, 0.01)
EOL

### Writing the README.md File

In [ ]:
%%writefile /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/README.md

# SappyCSTR

The `SappyCSTR` package is designed to enhance the conversion of reactants in a Continuous Stirred Tank Reactor (CSTR) engaged in a saponification reaction. Utilizing the Python libraries `Pyomo` for mathematical optimization and `Thermo` for thermodynamic property calculations, this package focuses on optimizing key operational parameters such as inlet temperature, inlet concentration of reactants, reactor volume, and inlet volumetric flow rate. By providing a structured and efficient approach to adjust these parameters, `SappyCSTR` aims to achieve optimal reactant conversion, thereby enhancing the efficiency and output of chemical processes within CSTR systems.

## Table of Contents
- [Installation](#installation)
- [Usage](#usage)
- [Examples](#examples)
- [Contributing](#contributing)
- [License](#license)

## Installation

Run the following line in a shell. For example, in a Bash shell:

'pip install SappyCSTR'

## Usage 

The primary usage of `SappyCSTR` is to optimize reactor conditions for saponification reactions occuring in a CSTR in adiabatic, isobaric conditions.  

First, import the function like so:

from SappyCSTR.core import optimize_CSTR

The inputs necessary to use the function `optimize_CSTR` are a dictionary of initial conditions, a rate constant at a reference temperature (scalar), activation energy for the reaction (scaler), and a dictionary defining the compound names, their role (reactant or product), and their stoichiometry. 

Here is the structure for initial_conditions:  

initial_conditions = {
    'C_A_in': **value**  # Inlet Concentration of A (M)
    'C_B_in': **value**,  # Inlet Concentration of B (M)
     'v0': **value**,# volumwtric flow rate (L/s)
     'V': **value**, # Reactor volume (L)
}

And here is the structure for compound_names:  

compound_names = {
    'reactants': {
        'A': {'formula': **'condensed structural formula'**, 'stoichiometry': **value**},  
        'B': {'formula': **'condensed structural formula'**, 'stoichiometry': **value**},   
    },
    'products': {
        'C': {'formula': **'condensed structural formula'**, 'stoichiometry': **value**},  # Ethanol
        'D': {'formula': **'condensed structural formula'**, 'stoichiometry': **value**},   # Sodium Acetate
    }
}

### Writing LICENSE  

Using the standard MIT License.

In [ ]:
%%writefile /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/LICENSE

Copyright <2024> <Jacob Medley>

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

In [ ]:
%%writefile /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/requirements.txt

pyomo==6.7.1
thermo==0.2.27

### Final Package Structure

In [ ]:
%%bash
cd /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/
tree

## Establishing Git Repo and Pushing and Publishing the Changes

### Establishing a Git Repo and Commiting Changes Locally

In [ ]:
# Clone the repository - skip if you already have the latest version locally
git clone https://github.com/jmedjpg/SappyCSTR.git
cd SappyCSTR

# Remove the unwanted files
git rm '.DS_Store' 'JacobMedleyProjectDraft...' # and so on for each file

# Commit the changes
git commit -m "Remove unwanted files"

# Push the changes to GitHub
git push origin main


In [ ]:
import os
from getpass import getpass
import subprocess

# Prompt for the GitHub token securely
token = getpass('Enter your GitHub token: ')

# Set the token as an environment variable
os.environ['GITHUB_TOKEN'] = token

# Set your project directory
project_dir = "/Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR"

# Define the commands to run in the shell
commands = [
    f"cd {project_dir}",
    "git init",
    "git config --global user.name 'Jacob Medley'",
    "git config --global user.email 'jmedley@andrew.cmu.edu'",
    "git add .",
    "git commit -m 'Wrote the inaugural versions of core.py, README.md, LICENSE, requirements.txt, setup.py, and test_core.py'",
    "git remote add origin https://github.com/jmedjpg/SappyCSTR.git",
    "git pull --rebase origin main",
    f"git push https://{os.environ['GITHUB_TOKEN']}@github.com/jmedjpg/SappyCSTR.git main"
]

# Execute the commands
for cmd in commands:
    process = subprocess.run(cmd, shell=True, text=True)
    if process.returncode != 0:
        print(f"Error executing command: {cmd}")
        break

# Clear the GitHub token from the environment
del os.environ['GITHUB_TOKEN']


In [ ]:
%%bash
cd /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR
git init
git add .
git commit -m "Wrote the inaugural versions of core.py, README.md, LICENSE, requirements.txt, setup.py, and test_core.py"

### Committing Changes to A Repository on my GitHub Account

In [ ]:
import os
from getpass import getpass

# Prompt for the token
token = getpass('Enter your GitHub token: ')

# Set the token as an environment variable
os.environ['GITHUB_TOKEN'] = token

In [ ]:
%%bash

export GITHUB_TOKEN=${GITHUB_TOKEN}

cd /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/
git init
git config --global user.name "Jacob Medley"
git config --global user.email "jmedley@andrew.cmu.edu"
git add .
git commit -m "Wrote the inaugural versions of core.py, README.md, LICENSE, requirements.txt, setup.py, and test_core.py"
git push https://${GITHUB_TOKEN}@github.com/jmedjpg/SappyCSTR.git main

### Publishing to PyPi

In [ ]:
%%bash
pip install setuptools wheel twine

cd /Users/jacob/Documents/CMU/Spring2024/JKMinis/CreatingScientificSoftware/Project/SappyCSTR/

rm -rf dist

python setup.py sdist bdist_wheel

twine upload dist/*


### Code Quality

In [ ]:
%%bash
flake8 SappyCSTR

### Importing the Package from Local Source

In [ ]:
compound_names = {
    'reactants': {
        'A': {'formula': 'C4H8O2', 'stoichiometry': 1},  # Ethyl Acetate
        'B': {'formula': 'NaOH', 'stoichiometry': 1},     # Sodium Hydroxide
    },
    'products': {
        'C': {'formula': 'C2H5OH', 'stoichiometry': 1},  # Ethanol
        'D': {'formula': 'NaC2H3O2', 'stoichiometry': 1},   # Sodium Acetate
    }
}

initial_conditions = {
    'C_A_in': 1.0,  # Inlet Concentration of A (M)
    'C_B_in': 1.0,  # Inlet Concentration of B (M)
     'v0': 0.5,# volumwtric flow rate (L/s)
     'V': 5, # Reactor volume (L)
}
rate_constant_303K = 0.18 # Example rate constant at 303K for ESTERIFICATION
activation_energy = 48.4*1000  # Example activation energy in J/mol for ESTERIFICATION